<a href="https://colab.research.google.com/github/MiguelUrenaPliego/GeoimageVision/blob/main/examples/example_position.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Position module example

This module computes the relative position of buildings

Install libraries if using colab

In [ ]:
!pip install geopandas
!pip install git+https://github.com/GeomaticsCaminosUPM/footprint_attributes.git
!pip install folium matplotlib mapclassify

  Cloning https://github.com/GeomaticsCaminosUPM/footprint_attributes.git to /tmp/pip-req-build-2jtt_jdh
  Running command git clone --filter=blob:none --quiet https://github.com/GeomaticsCaminosUPM/footprint_attributes.git /tmp/pip-req-build-2jtt_jdh
  Resolved https://github.com/GeomaticsCaminosUPM/footprint_attributes.git to commit b2fee43a7d47964ce00932e983511fb41d2884bd
  Preparing metadata (setup.py) ... done
  Created wheel for SeismicBuildingExposure: filename=SeismicBuildingExposure-0.1.0-py3-none-any.whl size=17367 sha256=977f32745454066cc6767b75002c584f04c2b82d770185c58428776c7d54dda9
  Stored in directory: /tmp/pip-ephem-wheel-cache-ul972oxr/wheels/7f/bd/c1/7a442bce743433d970ec52e18518fa8d82cb93ef44a7babcbf
Successfully built SeismicBuildingExposure
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
import footprint_attributes.position
import geopandas as gpd

Documentation of every function can be found using `help()` or under https://github.com/GeomaticsCaminosUPM/SeismicBuildingExposure

In [ ]:
help(footprint_attributes.position.contact_forces_df)

Help on function contact_forces_df in module footprint_attributes.position:

contact_forces_df(geoms: geopandas.geodataframe.GeoDataFrame, buffer: float = 0, height_column: str = None, min_radius: float = 0) -> pandas.core.frame.DataFrame
    Calculates force-based metrics for building footprints based on their geometry and proximity.
    
    Parameters:
        geoms (gpd.GeoDataFrame): A GeoDataFrame containing building footprints as polygon geometries.
        buffer (float): Buffer distance in meters to determine if two buildings are considered touching.
        height_column (str, optional): Column name in `geoms` specifying the building height in meters.
                                       If `None`, all buildings are assumed to have a uniform height of 1.
        min_radius (float, optional): Minimum distance multiplier used to exclude forces that would otherwise 
                                        increase momentum. Forces with a distance below a threshold 
           

## 1. Load data

Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

Download an example footprints file if needed

In [ ]:
!curl -L -o footprints.gpkg https://github.com/GeomaticsCaminosUPM/footprint_attributes/raw/refs/heads/main/examples/footprints.gpkg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  364k  100  364k    0     0   398k      0 --:--:-- --:--:-- --:--:-- 5350k


Load the footprints geodataframe in `.shp` `.gpkg` `.geojson` format using `geopandas.read_file()`

In [ ]:
footprints_gdf = gpd.read_file('footprints.gpkg')

- All geometries should be single part and polygons.

    - Multi part geometries (MultiPolygon) would mean two buildings are contained in the same footprint geometry which should not happen.

    - Footprints must be polygons, not linestrings.

Check if all geometries are `Polygon`

In [ ]:
if any(footprints_gdf.geometry.type == 'MultiPolygon'):
    print("There are multiplart geometries.")

if any(footprints_gdf.geometry.type.str.contains('Polygon') == False):
    print("Some geometries are not Polygon")

There are multiplart geometries.


Explode multipart geometries into single parts if needed


Note: The row number will be reset. Maybe you do not know how to match the footprints with your data. An id column is a good idea to solve this issue.

In [ ]:
footprints_gdf = footprints_gdf.explode().reset_index() # The new index column is the row number in the origninal dataframe
footprints_gdf

,index,gid,geometry
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1..."
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1..."
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1..."
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1..."
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1..."
...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1..."
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1..."
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1..."
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1..."


## 2. Relative position


Determines if the building touches other structures (relative position in the city block). This is done by calculating "contact forces" that neighboring structures exert on the building. The force is proportional to the contact area (length of touching footprints multiplied by building height) in the normal direction of the touching plane.


Lets calculate the forces with `contact_forces_df()`

In [ ]:
forces = footprint_attributes.position.contact_forces_df(footprints_gdf,height_column=None,buffer=0.2,min_radius=0.5) #20 cm of buffer
forces

,height,force,confinement_ratio,angular_acc,angle
0,1.0,0.697807,2.119091,0.320945,2.465851e+00
1,1.0,0.641808,0.766662,1.613179,1.159873e+00
2,1.0,0.341574,7.681600,0.001463,2.060583e-01
3,1.0,0.020213,179.790671,0.067755,2.266605e+00
4,1.0,0.287717,0.000000,0.220567,0.000000e+00
...,...,...,...,...,...
939,1.0,1.113140,0.000000,0.034521,0.000000e+00
940,1.0,0.791168,0.000000,0.220128,4.214685e-08
941,1.0,0.977905,1.111545,1.164580,1.587350e+00
942,1.0,0.769893,1.679563,0.817435,1.745959e+00


Now determine the relative position using the forces calculated before with `relative_position()`

In [ ]:
footprints_gdf['relative_position'] = footprint_attributes.position.relative_position(forces,min_angular_acc=2.133,min_confinement=1,min_angle=0.78,min_force=0.1666)
footprints_gdf

,index,gid,geometry,relative_position
0,0,1218.0,"POLYGON ((488600.239 1097577.417, 488591.045 1...",confined
1,1,1219.0,"POLYGON ((488562.593 1097585.096, 488567.403 1...",corner
2,2,1220.0,"POLYGON ((488601.331 1097581.438, 488599.351 1...",confined
3,3,1221.0,"POLYGON ((488517.642 1097593.605, 488517.592 1...",confined
4,4,1222.0,"POLYGON ((488492.697 1097591.081, 488492.546 1...",lateral
...,...,...,...,...
939,943,1273.0,"POLYGON ((488958.217 1097631.443, 488961.413 1...",lateral
940,944,1333.0,"POLYGON ((489119.978 1097677.125, 489130.663 1...",lateral
941,945,3632.0,"POLYGON ((489024.946 1097763.628, 489024.157 1...",confined
942,946,1505.0,"POLYGON ((488924.894 1097824.766, 488928.927 1...",confined


We create a new column `"relative_position"` that classifies buildings into the following categories:
  1. **"torque"**: Buildings of class **confined** or **corner** with an angular acceleration exceeding the minimum.
  2. **"confined"**: Structures touching on both the left and right lateral sides.
  3. **"corner"**: Structures touching at a corner (determined by force and angle thresholds).
  4. **"lateral"**: Structures touching on either the left or right side.
  5. **"isolated"**: No touching structures.

## 3. Example code to plot results


Plot a map (needs pip install folium matplotlib mapclassify)

In [ ]:
footprints_gdf.explore(
    column='relative_position',       # Column to base the color on
    cmap='gist_rainbow',        # Color map (Yellow to Red)
    legend=True ,           # Add a legend
)

## 4. Save results

In [ ]:
footprints_gdf.to_file('file.gpkg')